In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
%cd '/content/drive/My Drive/ModelSharing'

/content/drive/My Drive/ModelSharing


In [26]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import *
import tensorflow.keras.backend as K

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [27]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=False):
  n_vars = 1 if type(data) is list else data.shape[1]
  df = pd.DataFrame(data)
  cols, names = list(), list()
  # input sequence (t-n, ... t-1)
  for i in range(n_in, 0, -1):
    cols.append(df.shift(i))
    names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
  # forecast sequence (t, t+1, ... t+n)
  for i in range(0, n_out):
    cols.append(df.shift(-i))
    if i == 0:
      names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
    else:
      names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
  # put it all together
  agg = pd.concat(cols, axis=1)
  agg.columns = names
  # drop rows with NaN values
  if dropnan:
    agg.dropna(inplace=True)
  return pd.DataFrame(agg.astype('float32'))

In [28]:
def train_valid_test_split(data, hours_of_history, hours_to_predict, parameters_included):
  y_list = list(range(parameters_included*hours_of_history+parameters_included-1, parameters_included*(hours_of_history+hours_to_predict)+parameters_included-1, parameters_included))
  all_list = list(range(0, parameters_included*(hours_of_history+hours_to_predict)))
  x_list = [item for item in all_list if item not in y_list]

  data_train = data.iloc[:52608,:] # the first 6 years for training/validation
  data_test = data.iloc[52608:,:] # the last 1 years for test evaluation
  data_train.dropna(inplace=True)
  data_test.dropna(inplace=True)

  x_train_valid = data_train.iloc[:,x_list].values
  x_test = data_test.iloc[:,x_list].values
  y_train_valid = data_train.iloc[:,y_list].values
  y_test = data_test.iloc[:,y_list].values

  x_valid, x_train, y_valid, y_train = train_test_split(x_train_valid, y_train_valid, test_size=0.4, shuffle= False) # the first 60% data in the first 6 years used for training and the rest 40% used for validation.

  return x_train, x_valid, x_test, y_train, y_valid, y_test

In [29]:
def prepare_data(station_id, hours_of_history, hours_to_predict, parameters_included):
  # load the data
  data = pd.read_csv('./data/'+str(station_id)+'_data.csv').iloc[:,1:]

  # simple min-max scaling. Other pretreatments such as normalization also work.
  scaler = MinMaxScaler()
  scaler.fit(data.iloc[:52608,:]) # min-max scaling without the test dataset.
  q_max = np.max(data.iloc[:52608,2]) # manually check the maximum and minimum discharge
  q_min = np.min(data.iloc[:52608,2])
  data_scaled = scaler.transform(data)

  # data split
  data_sequence = series_to_supervised(data_scaled, hours_of_history, hours_to_predict)
  x_train, x_valid, x_test, y_train, y_valid, y_test = train_valid_test_split(data_sequence, hours_of_history, hours_to_predict, parameters_included)

  # Split data into 2 parts for encoder (history) and decoder(future).
  x_train_encoder = x_train[:,:hours_of_history*parameters_included].reshape(-1, hours_of_history, parameters_included)
  x_train_decoder = x_train[:,hours_of_history*parameters_included:].reshape(-1, hours_to_predict, parameters_included-1)
  x_valid_encoder = x_valid[:,:hours_of_history*parameters_included].reshape(-1, hours_of_history, parameters_included)
  x_valid_decoder = x_valid[:,hours_of_history*parameters_included:].reshape(-1, hours_to_predict, parameters_included-1)
  x_test_encoder = x_test[:,:hours_of_history*parameters_included].reshape(-1, hours_of_history, parameters_included)
  x_test_decoder = x_test[:,hours_of_history*parameters_included:].reshape(-1, hours_to_predict, parameters_included-1)

  return x_train_encoder, x_train_decoder, x_valid_encoder, x_valid_decoder, x_test_encoder, x_test_decoder, y_train, y_valid, y_test, q_max, q_min

In [30]:
# define custome loss function (you can use the simple 'mse' as well)
def nseloss(y_true, y_pred):
  return K.sum((y_pred-y_true)**2)/K.sum((y_true-K.mean(y_true))**2)

In [31]:
def NRM(hours_of_history, hours_to_predict, parameters_included):
  
  # design network
  dim_dense = [128, 64, 64, 32, 32]
  drop = 0.2

  encoder_input = Input(shape=(hours_of_history,parameters_included))
  encoder_GRU1 = GRU(32, return_state=True, return_sequences=True)
  encoder_output1, encoder_hc1 = encoder_GRU1(encoder_input)
  encoder_GRU2 = GRU(32, return_state=True, return_sequences=True)
  encoder_output2, encoder_hc2 = encoder_GRU2(encoder_output1)
  encoder_GRU3 = GRU(32, return_state=True, return_sequences=True)
  encoder_output3, encoder_hc3 = encoder_GRU3(encoder_output2)
  encoder_GRU4 = GRU(32, return_state=True, return_sequences=True)
  encoder_output4, encoder_hc4 = encoder_GRU4(encoder_output3)
  encoder_GRU5 = GRU(32, return_state=True)
  encoder_output5, encoder_hc5 = encoder_GRU5(encoder_output4)

  decoder_input = Input(shape=(hours_to_predict,parameters_included-1))
  decoder_GRU1 = GRU(32, return_sequences=True)
  decoder_GRU2 = GRU(32, return_sequences=True)
  decoder_GRU3 = GRU(32, return_sequences=True)
  decoder_GRU4 = GRU(32, return_sequences=True)
  decoder_GRU5 = GRU(32, return_sequences=True)
  x = decoder_GRU1(decoder_input, initial_state=encoder_hc1)
  x = decoder_GRU2(x, initial_state=encoder_hc2)
  x = decoder_GRU3(x, initial_state=encoder_hc3)
  x = decoder_GRU4(x, initial_state=encoder_hc4)
  x = decoder_GRU5(x, initial_state=encoder_hc5)

  for dim in dim_dense:
    x = TimeDistributed(Dense(dim, activation='tanh'))(x) # or relu
    x = TimeDistributed(Dropout(drop))(x)
  main_out = TimeDistributed(Dense(1, activation='linear'))(x) # relu as the last activation works (only) for the min-max scalling so there is no negative output
  main_out = Flatten()(main_out)
  model = Model(inputs=[encoder_input, decoder_input], outputs=main_out)
  return model

In [32]:
# identify KGE, NSE for evaluation
def nse(y_true, y_pred):
  return 1-np.sum((y_pred-y_true)**2)/np.sum((y_true-np.mean(y_true))**2)
  
def kge(y_true, y_pred):
  kge_r = np.corrcoef(y_true,y_pred)[1][0]
  kge_a = np.std(y_pred)/np.std(y_true)
  kge_b = np.mean(y_pred)/np.mean(y_true)
  return 1-np.sqrt((kge_r-1)**2+(kge_a-1)**2+(kge_b-1)**2)


In [33]:
def main():
  # parameters
  station_id = 521
  hours_to_predict = 120
  hours_of_history = 72
  parameters_included = 3

  batch_size = 64
  lr = 0.0001
  epochs = 300
  test_name = './'+str(station_id)+'_model2_'

  # load data
  x_train_encoder, x_train_decoder, x_valid_encoder, x_valid_decoder, x_test_encoder, x_test_decoder, y_train, y_valid, y_test, q_max, q_min = prepare_data(station_id, hours_of_history, hours_to_predict, parameters_included)
  model2 = NRM(hours_of_history, hours_to_predict, parameters_included)

  # compile settings
  reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, cooldown=200, min_lr=1e-8)
  earlystoping = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1, mode='auto')
  checkpoint = ModelCheckpoint(test_name+'model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
  optimizer = RMSprop(lr=lr)
  model2.compile(optimizer=optimizer, loss=nseloss) # in paper, we used the customized "nseloss" as model 1 did. however, here we show mse as an example.

  # train model
  history = model2.fit([x_train_encoder, x_train_decoder], y_train, epochs=epochs, batch_size=batch_size,
              validation_data=([x_valid_encoder, x_valid_decoder], y_valid), callbacks=[reduce_lr, earlystoping, checkpoint], verbose=1)

  # save training loss
  loss_train = history.history['loss']
  loss_valid = history.history['val_loss']
  loss_train = pd.DataFrame({'TrainLoss':loss_train})
  loss_valid = pd.DataFrame({'TestLoss':loss_valid})
  LossEpoches = pd.concat([loss_train, loss_valid], axis=1)
  LossEpoches.to_csv(test_name+'loss.csv', index = True)

  # Final Test Review
  model2.load_weights(test_name+'model.h5')

  y_model_scaled = model2.predict([x_test_encoder,x_test_decoder])
  y_model = y_model_scaled*(q_max-q_min)+q_min
  y_test = y_test*(q_max-q_min)+q_min

  # hourly evaluation
  NSEs=[]
  KGEs=[]
  for x in range(0, 120):
    y_pred = y_model[:,x]
    y_True = y_test[:,x]
    NSEs.append(nse(y_True,y_pred))
    KGEs.append(kge(y_True,y_pred))
    
  NSEs=pd.DataFrame(NSEs)
  NSEs.columns = ['NSE_Test']
  KGEs=pd.DataFrame(KGEs)
  KGEs.columns = ['KGE_Test']
    
  eva = pd.concat([NSEs, KGEs], axis=1)
  eva.to_csv(test_name+'eva.csv', index = True)
 

In [34]:
if __name__ == "__main__":
  main()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Epoch 1/300
273/274 [============================>.] - ETA: 0s - loss: 1.0383
Epoch 00001: val_loss improved from inf to 5531.61035, saving model to ./521_model2_model.h5
274/274 [==============================] - 20s 73ms/step - loss: 1.0383 - val_loss: 5531.6104
Epoch 2/300
273/274 [============================>.] - ETA: 0s - loss: 0.8461
Epoch 00002: val_loss improved from 5531.61035 to 3695.69385, saving model to ./521_model2_model.h5
274/274 [==============================] - 18s 66ms/step - loss: 0.8463 - val_loss: 3695.6938
Epoch 3/300
273/274 [============================>.] - ETA: 0s - loss: 0.7912
Epoch 00003: val_loss did not improve from 3695.69385
274/274 [==============================] - 18s 65ms/step - loss: 0.7910 - val_loss: 5432.3809
Epoch 4/300
273/274 [============================>.] - ETA: 0s - loss: 0.7527
Epoch 00004: val_loss did not improve from 3695.69385
274/274 [==============================] - 18s 65ms/step - loss: 0.7529 - val_loss: 6673.6909
Epoch 5/300